In [1]:
import pandas as pd

# Assignment 1: Groupby

#### Top Stores by Transactions (Merchandising)

Can you return a table containing the top 10 stores by total transactions in the data?

Make sure they’re sorted from highest to lowest.

Thanks!

In [2]:
# Read in transactions data -- parse dates specified here for help with later problem

transactions = pd.read_csv("../retail/transactions.csv", parse_dates=["date"])

transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [3]:
# Group transactions by store_nbr number. Perform sum aggregation on transactions column

(transactions
 .groupby(["store_nbr"])[["transactions"]] 
 .sum() 
 .sort_values("transactions", ascending=False)  # sort in descending order
 .iloc[:10]  # Grab top 10 rows (will be highest values due to sort descending)
)

,transactions
store_nbr,
44,7273093
47,6535810
45,6201115
46,5990113
3,5366350
48,5107785
8,4637971
49,4574103
50,4384444


# Assignment 2: Groupby Multiple Columns

#### Product (Merchandising) - Transactions by Store and Month

Taking this analysis a layer deeper... Can you get me the total transactions by store and month? 

Sort the table from first month to last, then by highest transactions to lowest within each month. 

I'll likely analyze this data further in a dashboard software, but this will help me set up special seasonal displays


In [4]:
# helper code to extract month date part from date column

transactions["month"] = transactions["date"].dt.month

transactions.head()

,date,store_nbr,transactions,month
0,2013-01-01,25,770,1
1,2013-01-02,1,2111,1
2,2013-01-02,2,2358,1
3,2013-01-02,3,3487,1
4,2013-01-02,4,1922,1


In [5]:
# group by store number and month, sum trasactions

(transactions
 .groupby(["store_nbr", "month"])[["transactions"]]
 .sum()
 .sort_values(["month", "transactions"], ascending=[True, False] # ascending month, descending transactions
))

,,transactions
store_nbr,month,
44,1,628438
47,1,568824
45,1,538370
46,1,522763
3,1,463260
...,...,...
32,12,86167
21,12,84128
42,12,76741


# Assignment 3: Multi-Index DataFrames

Multi-Index Help! (Head of Analytics)

Can you help me access rows and columns with multiple indices? I’ve been struggling with multi-index DataFrames.

Access:
* Grab Store 3, Month 1
* Then, select the column storing the mean of transactions

Fix:
* Drop the outer layer of the column Index
* Reset the row index so it is the default integer index

In [6]:
# Ross' grouped DataFrame code, run this first

grouped = (
    transactions.groupby(["store_nbr", "month"])
    .agg({"transactions": ["sum", "mean"]})
    .sort_values(by=["month", ("transactions", "sum")], ascending=[True, False])
)

In [7]:
# df grouped by store and month with two aggregation columns

grouped.head()

transactions             
                         sum         mean
store_nbr month                          
44        1           628438  4246.202703
47        1           568824  3843.405405
45        1           538370  3637.635135
46        1           522763  3532.182432
3         1           463260  3151.428571

In [8]:
# Grab store 3, month 1 from multi-index (both values are integers)

grouped.loc[(3, 1)] # equivalent to grouped.iloc[4]

transactions  sum     463260.000000
              mean      3151.428571
Name: (3, 1), dtype: float64

In [9]:
# Grab mean column in column multi-index

grouped.loc[:, [("transactions", "mean")]] 

# equivalent to grouped.iloc[:, 1] (all rows, second column)
# wrap the integer in brackets return a dataframe: grouped.iloc[:, [1]]

,,transactions
,,mean
store_nbr,month,
44,1,4246.202703
47,1,3843.405405
45,1,3637.635135
46,1,3532.182432
3,1,3151.428571
...,...,...
32,12,718.058333
21,12,1402.133333


In [10]:
# Drop level from column index (axis=1), then reset index
# Drop "transactions" level from column index, then reset index

grouped.droplevel(0, axis=1).reset_index()

,store_nbr,month,sum,mean
0,44,1,628438,4246.202703
1,47,1,568824,3843.405405
2,45,1,538370,3637.635135
3,46,1,522763,3532.182432
4,3,1,463260,3151.428571
...,...,...,...,...
636,32,12,86167,718.058333
637,21,12,84128,1402.133333
638,42,12,76741,1279.016667
639,29,12,76627,1277.116667


# Assignment 4: The Agg Method

#### Bonus Rate and Bonus Payable (Accounting)

Calculate the mean of target met by store, and the sum of bonuses to be paid to each store.

Sort them by highest to lowest bonus payout.

Then, do the same for day of week and month.

In [11]:
# Recreate table from section 3

transactions = transactions.assign(
    target_pct=transactions["transactions"] / 2500,
    met_target=(transactions["transactions"] / 2500) >= 1,
    bonus_payable=((transactions["transactions"] / 2500) >= 1) * 100,
    month=transactions.date.dt.month,
    day_of_week=transactions.date.dt.dayofweek,
)

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


_By Store:_

In [12]:
# Group transactions by store number to get store level stats 
# use agg to calculate mean of met_target, sum of transactions by store
# sort values by bonus payable in descending order

(transactions.groupby("store_nbr")
 .agg({"met_target": "mean", "bonus_payable": "sum"})
 .sort_values(by=["bonus_payable"], ascending=False) # equivalent to .sort_values("bonus_payable", ascending=False) 
)

,met_target,bonus_payable
store_nbr,,
47,0.999404,167600
44,0.998807,167500
45,0.997615,167300
3,0.998210,167300
46,0.989267,165900
8,0.888425,148900
48,0.690519,115800
49,0.637448,106900
50,0.453190,76000


_By Month:_

In [13]:
# Group transactions by month to get month level stats 
# use agg to calculate mean of met_target, sum of transactions by store
# sort values by bonus payable in descending order

(transactions.groupby("month")
 .agg({"met_target": "mean", "bonus_payable": "sum"})
 .sort_values(by=["bonus_payable"], ascending=False)
)

,met_target,bonus_payable
month,,
12,0.255640,154100
5,0.170792,131800
3,0.169461,130400
4,0.174469,129700
7,0.162486,126300
2,0.174230,121700
6,0.161706,121700
8,0.174189,120800
1,0.163723,119600


_By Weekday:_

In [14]:
# Group transactions by day of week to get day of week level stats 
# use agg to calculate mean of met_target, sum of transactions by store
# sort values by bonus payable in descending order

(transactions
 .groupby(["day_of_week"])
 .agg({"met_target": "mean", "bonus_payable": "sum"})
 .sort_values("bonus_payable", ascending=False)
)

,met_target,bonus_payable
day_of_week,,
5,0.222204,266400
6,0.204001,241700
4,0.179007,213000
0,0.160214,191600
2,0.160572,191000
1,0.146299,175500
3,0.142077,169100


# Assignment 4: Transform

#### Store Transactions Vs. Average (Accounting)

Calculate the mean of transactions by store number and day of week while keeping row numbers. 

Then compare the performance of each row to its day of week average. (difference between transactions and daily avg)

In [15]:
# Use trasnactions table created in Assignment 3

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


In [16]:
# Assign method for column creation

transactions.assign(
    # use transform to create store level average transactions by day
    avg_store_transactions = (transactions
                              .groupby(["store_nbr", "day_of_week"])
                              ["transactions"]
                              .transform("mean")),
    
    # Then create a difference column subtracting the store average from that days transactions
    difference = lambda x: x["transactions"] - x["avg_store_transactions"] # x is our dataframe
)

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week,avg_store_transactions,difference
0,2013-01-01,25,770,1,0.3080,False,0,1,740.245690,29.754310
1,2013-01-02,1,2111,1,0.8444,False,0,2,1870.782427,240.217573
2,2013-01-02,2,2358,1,0.9432,False,0,2,1952.652720,405.347280
3,2013-01-02,3,3487,1,1.3948,True,100,2,3142.682008,344.317992
4,2013-01-02,4,1922,1,0.7688,False,0,2,1499.569038,422.430962
...,...,...,...,...,...,...,...,...,...,...
83483,2017-08-15,50,2804,8,1.1216,True,100,1,2342.410788,461.589212
83484,2017-08-15,51,1573,8,0.6292,False,0,1,1548.448133,24.551867
83485,2017-08-15,52,2255,8,0.9020,False,0,1,1892.588235,362.411765
83486,2017-08-15,53,932,8,0.3728,False,0,1,877.214286,54.785714


In [17]:
# Take a peek at store level aggregate values for store 25 to validate column

(transactions
 .query("store_nbr == 25")
 .groupby(["day_of_week"])
 .agg({"transactions": "mean"})
)

,transactions
day_of_week,
0,808.830435
1,740.245690
2,849.978261
3,800.818182
4,1102.817391
5,1376.090517
6,909.604348


# Assignment 5: Pivot

Pivot transactions with store number as index, columns day of week, with the sum of bonus payable as cells.

Filter to stores that had a non-zero bonus payable and create a heatmap.

Then unpivot (melt) the table so we have one row for each store and day of the week with the corresponding total owed. 


In [18]:
# Use transactions table (ok if includes columns from assignment 4 or not)

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


In [19]:
# Filter out all rows with 0 bonus. 

(transactions.loc[transactions["bonus_payable"] != 0]
.pivot_table(
    index="store_nbr",       # store number as rows
    columns="day_of_week",   # day of week as columns
    values="bonus_payable",  # aggregate bonus payable column
    aggfunc="sum")           # sum bonus payable
.iloc[:10]                   # optional filter to reduce table size
.style.background_gradient(cmap="RdYlGn", axis=1 ))  # apply RdYlGn colormap to pivoted DF; 
                                                     # axis=1 to apply to columns (where are the hot spots for each store)

day_of_week,0,1,2,3,4,5,6
store_nbr,,,,,,,
1,200.000000,300.000000,300.000000,200.000000,100.000000,nan,nan
2,300.000000,600.000000,500.000000,400.000000,400.000000,500.000000,200.000000
3,24000.000000,23900.000000,23900.000000,23900.000000,23900.000000,24000.000000,23700.000000
4,200.000000,300.000000,300.000000,200.000000,100.000000,200.000000,nan
5,200.000000,300.000000,300.000000,100.000000,100.000000,100.000000,nan
6,400.000000,500.000000,500.000000,300.000000,200.000000,900.000000,300.000000
7,200.000000,300.000000,300.000000,200.000000,100.000000,100.000000,nan
8,22000.000000,18800.000000,23800.000000,18000.000000,22900.000000,23400.000000,20000.000000
9,1200.000000,800.000000,800.000000,700.000000,400.000000,7900.000000,5100.000000


In [21]:
(transactions.loc[transactions["bonus_payable"] != 0]
.pivot_table(
    index="store_nbr",       # store number as rows
    columns="day_of_week",   # day of week as columns
    values="bonus_payable",  # aggregate bonus payable column
    aggfunc="sum")           # sum bonus payable
.iloc[:10]                   # optional filter to reduce table size
.style.background_gradient(cmap="Blues", axis=1 )) 

day_of_week,0,1,2,3,4,5,6
store_nbr,,,,,,,
1,200.000000,300.000000,300.000000,200.000000,100.000000,nan,nan
2,300.000000,600.000000,500.000000,400.000000,400.000000,500.000000,200.000000
3,24000.000000,23900.000000,23900.000000,23900.000000,23900.000000,24000.000000,23700.000000
4,200.000000,300.000000,300.000000,200.000000,100.000000,200.000000,nan
5,200.000000,300.000000,300.000000,100.000000,100.000000,100.000000,nan
6,400.000000,500.000000,500.000000,300.000000,200.000000,900.000000,300.000000
7,200.000000,300.000000,300.000000,200.000000,100.000000,100.000000,nan
8,22000.000000,18800.000000,23800.000000,18000.000000,22900.000000,23400.000000,20000.000000
9,1200.000000,800.000000,800.000000,700.000000,400.000000,7900.000000,5100.000000


In [20]:
# Build pivot table from above then melt

(transactions
 .loc[transactions["bonus_payable"] != 0]
 .pivot_table(index="store_nbr",
              columns="day_of_week",
              values="bonus_payable",
              aggfunc="sum")
 .reset_index()  # get necessary columns out of index before melting
 .melt(id_vars="store_nbr", value_name="bonus_payable"))  # specify store_nbr to melt around and name bonus column

,store_nbr,day_of_week,bonus_payable
0,1,0,200.0
1,2,0,300.0
2,3,0,24000.0
3,4,0,200.0
4,5,0,200.0
...,...,...,...
240,49,6,21600.0
241,50,6,23200.0
242,51,6,200.0
243,52,6,1500.0
